# Installing Packages and Libraries

In [1]:
!pip install rake_nltk

import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Ingestion script

#Reading data directly from multiple sheets and concatinating it in a single dataframe.
Changed names to accumulate clerical errors of leading and trailing spaces.
Dropping (un)wanted columns from the data for initial testing.
Changing NaN values to blank for now.


In [2]:
menus=pd.concat(pd.read_excel('Chef Menus complete.xlsx',sheet_name=None,na_values=['TBD']),ignore_index=False)
menus.reset_index(level=0, inplace=True)
menus.rename(columns={'level_0':'chefname',
                       'Course 1':'course_1',
                       'Course 2':'course_2',
                       'Course 3 ':'course_3',
                       'Course 4' :'course_4',
                       'Menu Description ':'Menu_description',
                     'Course 1 Description' : 'Course1_Description', 
             'Course 1 Description' : 'Course1_Description',
                         'Course 2 Description' : 'Course2_Description',
                         'Course 3 Description' : 'Course3_Description',
                         'Course 4 Description' :  'Course4_Description'},
                 inplace=True)
from nltk.corpus import stopwords
print(stopwords.words('english'))



#menus.info(verbose=True)
#menus.drop(['chefname','Menu_description','Cuisine Type','Dates available to cook this menu','course_1','course_2','course_3','course_4','course_summary','Price'], axis = 1,inplace = True)
menus.drop(['chefname','Menu_description','Cuisine Type','Dates available to cook this menu','Price'], axis = 1,inplace = True) 
#print(menus.columns)
menus = menus.set_index('Menu Name')
menus = menus.replace(np.nan, '', regex=True)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Code below can be used to clean each menu description and create a key words out of each description.

In [3]:
# menus["course_summary"] = menus["course_1"].astype(str) + menus["course_2"].astype(str) + menus["course_3"].astype(str) + menus["course_4"].astype(str)
# menus['Key_words'] = ""
# menus['key']=''
# keys=0
# for index,rows in menus.iterrows():
#     course_summary = rows['course_summary']
#     r = Rake()
#     r.extract_keywords_from_text(course_summary)
#     key_words_dict_scores = r.get_word_degrees()
#     #print(key_words_dict_scores)
#     rows['key']=keys
#     rows['Key_words'] = list(key_words_dict_scores.keys())
#     keys=keys+1

# Data Cleaning

#Converting all values to comma seperated values & lower case for furthur process

In [4]:
menus['course_1']=menus['course_1'].astype(str).map(lambda x: x.lower().split(','))
menus['course_2']=menus['course_2'].astype(str).map(lambda x: x.lower().split(','))
menus['course_3']=menus['course_3'].astype(str).map(lambda x: x.lower().split(','))
menus['course_4']=menus['course_4'].astype(str).map(lambda x: x.lower().split(','))
menus['Course1_Description']=menus['Course1_Description'].astype(str).map(lambda x: x.lower().split(','))
menus['Course2_Description']=menus['Course2_Description'].astype(str).map(lambda x: x.lower().split(','))
menus['Course3_Description']=menus['Course3_Description'].astype(str).map(lambda x: x.lower().split(','))
menus['Course4_Description']=menus['Course4_Description'].astype(str).map(lambda x: x.lower().split(','))



Concatinating all the columns(Course 1 , 2 ,3 ,4) into a single "bag of words".

In [5]:
menus['Course1_bag_of_words'] = ''
columns = ['course_1','Course1_Description']
menus['Course1_bag_of_words'] = menus['course_1']+menus['Course1_Description']
for index, row in menus.iterrows():
    words = ''
    for col in columns:
        words = words + ' '.join(row[col])+ ' '
    row['Course1_bag_of_words'] = words 
    
    
columns = ['course_2','Course2_Description']

menus['Course2_bag_of_words'] = menus['course_2']+menus['Course2_Description']
for index, row in menus.iterrows():
    words = ''
    for col in columns:
        words = words + ' '.join(row[col])+ ' '
    row['Course2_bag_of_words'] = words  
    
columns = ['course_3','Course3_Description']
menus['Course3_bag_of_words'] = menus['course_3']+menus['Course3_Description']
    
for index, row in menus.iterrows():
    words = ''
    for col in columns:
         words = words + ' '.join(row[col])+ ' '
    row['Course3_bag_of_words'] = words  
    
columns = ['course_4','Course4_Description']
menus['Course4_bag_of_words'] = menus['course_4']+menus['Course4_Description']
for index, row in menus.iterrows():
    words = ''
    for col in columns:
         words = words + ' '.join(row[col])+ ' '
    row['Course4_bag_of_words'] = words  

menus.drop(['course_1','course_2','course_3','course_4','Course1_Description','Course2_Description','Course3_Description','Course4_Description'],axis=1,inplace = True)


# Recommendation Algorithm

Count vectorizer is used to convert a collection of text documents to a vector of term/token counts. The reason for not using TF-IDF here because TF-IDF reduces the importance of a word if it occurs frequently.

Cosine similarity is a metric used to measure how similar the documents are irrespective of their size. Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space. 

Forming a similarity matrix here to get similarity scores.

In [9]:
course_1_weight=0.2
course_2_weight=0.3
course_3_weight=0.3
course_4_weight=0.2


In [10]:
count_CV = CountVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
count_matrix_CV_1 = count_CV.fit_transform(menus['Course1_bag_of_words']) 
count_matrix_CV_2 = count_CV.fit_transform(menus['Course2_bag_of_words'])
count_matrix_CV_3 = count_CV.fit_transform(menus['Course3_bag_of_words'])
count_matrix_CV_4 = count_CV.fit_transform(menus['Course4_bag_of_words'])


In [11]:
count_IDF = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
count_matrix_IDF_1 = count_IDF.fit_transform(menus['Course1_bag_of_words']) 
count_matrix_IDF_2 = count_IDF.fit_transform(menus['Course2_bag_of_words'])
count_matrix_IDF_3 = count_IDF.fit_transform(menus['Course3_bag_of_words'])
count_matrix_IDF_4 = count_IDF.fit_transform(menus['Course4_bag_of_words'])

In [12]:
cosine_sim_CV_1 = cosine_similarity(count_matrix_CV_1, count_matrix_CV_1) * course_1_weight
cosine_sim_CV_2 = cosine_similarity(count_matrix_CV_2, count_matrix_CV_2) * course_2_weight
cosine_sim_CV_3 = cosine_similarity(count_matrix_CV_3, count_matrix_CV_3) * course_3_weight
cosine_sim_CV_4 = cosine_similarity(count_matrix_CV_4, count_matrix_CV_4) * course_4_weight

cosine_sim_CV = cosine_sim_CV_1 + cosine_sim_CV_2 + cosine_sim_CV_3 + cosine_sim_CV_4
indices = pd.Series(menus.index)
cosine_sim_CV

array([[0.8       , 0.02188236, 0.01762246, ..., 0.        , 0.        ,
        0.        ],
       [0.02188236, 1.        , 0.10894716, ..., 0.        , 0.        ,
        0.        ],
       [0.01762246, 0.10894716, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.6       , 0.        ,
        0.14142136],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.14142136, 0.        ,
        0.3       ]])

In [15]:
cosine_sim_IDF_1 = cosine_similarity(count_matrix_IDF_1, count_matrix_IDF_1) * course_1_weight
cosine_sim_IDF_2 = cosine_similarity(count_matrix_IDF_2, count_matrix_IDF_2) * course_2_weight
cosine_sim_IDF_3 = cosine_similarity(count_matrix_IDF_3, count_matrix_IDF_3) * course_3_weight
cosine_sim_IDF_4 = cosine_similarity(count_matrix_IDF_4, count_matrix_IDF_4) * course_4_weight

cosine_sim_IDF = cosine_sim_IDF_1 + cosine_sim_IDF_2 + cosine_sim_IDF_3 + cosine_sim_IDF_4
cosine_sim_IDF

array([[0.8       , 0.00798626, 0.00431706, ..., 0.        , 0.        ,
        0.        ],
       [0.00798626, 1.        , 0.04121454, ..., 0.        , 0.        ,
        0.        ],
       [0.00431706, 0.04121454, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.6       , 0.        ,
        0.12417142],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.12417142, 0.        ,
        0.3       ]])

# Creating a function to return top recommended menu items.


In [22]:
def recommendations(menu, top, weight_TFIDF):
    # Creating an empty recommended menu list
    weight_CV=1-weight_TFIDF
    cosine_sim_mixed = cosine_sim_IDF * weight_TFIDF + cosine_sim_CV * weight_CV
    recommended_menu = []
    # gettin the index of provided user menu that matches the menu
    idx = indices[indices == menu].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim_mixed[idx]).sort_values(ascending = False)
    # getting the indexes of the 3 most similar menu
    top_3_indexes = list(score_series.iloc[1:top+1].index)
    print('Top '+str(top)+' recommendation for '+menu+'------->')
    # populating the list with the titles of the best 3 matching menus
    for i in top_3_indexes:
        recommended_menu.append(list(menus.index)[i])
        
    return recommended_menu


# Testing

Testing above function with dummy data.

In [27]:
recommendations(menu='Everything Seafood',top=3,weight_TFIDF= 0.5)

Top 3 recommendation for Everything Seafood------->


['gt', 'For the Love of Vegetable', 'Mediterranean']

In [28]:
recommendations(menu='Pasta ',top=3,weight_TFIDF= 0.5)

Top 3 recommendation for Pasta ------->


['Mediterranean', 'gt', 'Everything Seafood']